# Summarizing Customer Support Cases with LLMs, Langchain and ServiceNow

This Jupyter Notebook accompanies the blog post: <insert blogpost> and provides a simple example of using the OpenAI gpt 3.5 turbo LLM to generate a summary for ServiceNow Customer Support Cases.

In [ ]:
! python -m pip install openai langchain python-dotenv requests

**Environment variables**

We need to set an evnironment variable to specify your OpenAI API Key.
You can edit the code below to set them directly.

* `OPENAI_API_KEY` - Your OpenAI API key

If a local `.env` file exists, load the env vars from it.





In [ ]:
from dotenv import load_dotenv
load_dotenv()

False

In [ ]:
import os

print("Check environment\n---------------------")
openai_api_key = os.environ.get('OPENAI_API_KEY') or "YOUR OPENAI API KEY"
print("openai_api_key:", openai_api_key[:5], "...")


Check environment
---------------------
openai_api_key: YOUR  ...


If your output looks similar to this, then you're ready to go!

openai_api_key: sk-7w ...

In [ ]:
import json
import requests
import base64
from langchain import PromptTemplate
import openai

openai.api_key = openai_api_key

qa_template_str = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context:
-----------------
Customer Support Case{context}
-----------------

Question: {question}
Short Answer:"""
qa_template = PromptTemplate(template=qa_template_str, input_variables=["context", "question"])

def run_llm_qa_prompt(context: str, question: str):
    qa_prompt = qa_template.format(context=context, question=question)
    print(">>> Prompt Start >>>>>>>>>>>>>>>>>>>>>>>>>>>")
    print(qa_prompt)
    print("<<< Prompt End <<<<<<<<<<<<<<<<<<<<<<<<<<<<<")

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "system", "content": qa_prompt}],
        temperature=0.0
    )

    answer = response['choices'][0]['message']['content'].strip()
    print("\nLLM Response:", answer)
    return answer

In [ ]:
# example customer case from ServiceNow
customer_case = {
        "sys_updated_on": "2023-09-15 11:01:05",
        "number": "CS0001014",
        "initiated_as_request": "false",
        "state": "Resolved",
        "case": "Performance Issue when downloading Reports CS0001026",
        "sys_created_by": "admin",
        "impact": "3 - Low",
        "active": "true",
        "auto_created_case": "false",
        "priority": "1 - Critical",
        "first_response_time": "2023-09-11 02:25:24",
        "needs_attention": "false",
        "short_description": "Analytics Suite Blackout",
        "assigned_to": "Emily Doe",
        "product": "ACME Corporation Analytics Suite 2016",
        "comments_and_work_notes": "2023-09-15 14:53:07 - System (Additional comments)\nPlease follow this link to see a detailed review of the case: [code]<a href=\"/sys_attachment.do?sys_id=ca04f000db19330092daf3c61d961984\" target=\"_blank\">Post Case Review - 2019-05-03 21:53:06.pdf</a>[/code].\n\n2023-09-05 20:08:55 - Beth Anglin (Work notes)\nAdded as a child of CS0001015\n\n2023-09-05 20:07:40 - Emily Jason (Work notes)\nProposed as major case candidate\nMultiple Customers affected by the Blackout\n\n",
        "asset": "ACME Corporation Analytics Suite 2016",
        "made_sla": "true",
        "task_effective_number": "CS0001026",
        "resolved_by": "System Administrator",
        "sys_updated_by": "system",
        "opened_by": "System Administrator",
        "sys_created_on": "2023-09-05 22:11:53",
        "contact": "Sam Doe",
        "subcategory": "Question",
        "work_notes": "2023-09-05 20:08:55 - Beth Anglin (Work notes)\nAdded as a child of CS0001015\n\n2023-09-05 20:07:40 - Emily Jason (Work notes)\nProposed as major case candidate\nMultiple Customers affected by the Blackout\n\n",
        "assignment_group": "ACME Support",
        "close_notes": "Issue resolved.",
        "sys_id": "289276b4db4d330092daf3c61d9619d2",
        "contact_type": "Web",
        "resolution_code": "Solved – Fixed by Support/Guidance provided",
        "probable_cause": "",
        "approval": "Not Yet Requested",
        "sys_mod_count": "12",
        "internal_user": "",
        "account": "ACME"
    }





In [ ]:
# OPTIONAL CODE TO GET THE JSON of an actual Case from ServiceNow using the Table API
# specify servicenow instance_id and credentials to use when calling the ServiceNow table API
sn_uid = os.environ.get('SERVICENOW_USER_ID') or "YOUR SERVICENOW USER ID"
sn_pwd = os.environ.get('SERVICENOW_PWD') or "YOUR SERVICENOW INSTANCE PASSWORD"
sn_instance_id = os.environ.get('SERVICENOW_INSTANCE_ID') or "YOUR SERVICENOW INSTANCE ID"

print("Servicenow instance connection details:")
print("sn_uid:", sn_uid)
print("sn_pwd:", sn_pwd[:5], "...")
print(f"sn_instance_id: {sn_instance_id}\n")

url = "https://" + sn_instance_id + ".service-now.com/api/now/table/sn_customerservice_case?sysparm_display_value=true&sysparm_exclude_reference_link=true&number=CS0001014"
# generate auth header
auth_str = f"{sn_uid}:{sn_pwd}"
auth_string_bytes = auth_str.encode("ascii")
auth_string_bytes_b64 = base64.b64encode(auth_string_bytes)
auth_header = f"Basic {str(auth_string_bytes_b64, encoding='utf-8')}"

payload = {}
headers = {"Authorization": auth_header}

response = requests.request("GET", url, headers=headers, data=payload)
if response.status_code == requests.codes.ok:
  print(f'Request to ServiceNow API Successful, returned 200 OK\n\n')
else:
  print(f"Failed making request to ServiceNow API, returned {response.status_code}. Check to make sure your instance is awake and accessible via the web portal.\n\n")

customer_case = response.json()["result"][0]
customer_case

In [ ]:
# set the ServiceNow Customer Service Case as context
context = json.dumps(customer_case)

# instruct the LLM what you want it to do and how you would like your output formatted
question = "please summarize the work notes in two sentences and include the number and state field in your answer"

# generate summary from llm
run_llm_qa_prompt(context, question)
